In [1]:
!pip install -q transformers==4.29 datasets==2.14.5 sentencepiece==0.1.99

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "

In [2]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch

import numpy as np
import logging
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## Load the Model

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Set device to GPU if available

model_cp = "marefa-nlp/marefa-ner"
custom_labels = ["O", "B-job", "I-job", "B-nationality", "B-person", "I-person", "B-location",
                 "B-time", "I-time", "B-event", "I-event", "B-organization", "I-organization",
                 "I-location", "I-nationality", "B-product", "I-product", "B-artwork", "I-artwork"]

# Move model to GPU
tokenizer = AutoTokenizer.from_pretrained(model_cp)
model = AutoModelForTokenClassification.from_pretrained(model_cp, num_labels=len(custom_labels)).to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:454: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

In [4]:
def _extract_ner(text: str, model: AutoModelForTokenClassification,
                 tokenizer: AutoTokenizer, start_token: str="▁"):

    # Tokenize and move to GPU
    tokenized_sentence = tokenizer([text], padding=True, truncation=True, return_tensors="pt").to(device)


    with torch.no_grad():
        output = model(**tokenized_sentence)

    # Move outputs to CPU for further processing
    last_hidden_states = output.logits.cpu().numpy()
    label_indices = np.argmax(last_hidden_states[0], axis=1)

    tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence["input_ids"].cpu().numpy()[0])
    special_tags = set(tokenizer.special_tokens_map.values())

    grouped_tokens = []
    for token, label_idx in zip(tokens, label_indices):
        if token not in special_tags:
            if not token.startswith(start_token) and len(token.replace(start_token,"").strip()) > 0:
                grouped_tokens[-1]["token"] += token
            else:
                grouped_tokens.append({"token": token, "label": custom_labels[label_idx]})

    # Extract entities
    ents = []
    prev_label = "O"
    for token in grouped_tokens:
        label = token["label"].replace("I-","").replace("B-","")
        if token["label"] != "O":
            if label != prev_label:
                ents.append({"token": [token["token"]], "label": label})
            else:
                ents[-1]["token"].append(token["token"])

        prev_label = label

    # Group tokens
    ents = [{"token": "".join(rec["token"]).replace(start_token," ").strip(), "label": rec["label"]}  for rec in ents]

    return ents

## Model Inference

In [6]:
samples = [
    "تلقى تعليمه في الكتاب ثم انضم الى الأزهر عام 1873م. تعلم على يد السيد جمال الدين الأفغاني والشيخ محمد عبده",
    "بعد عودته إلى القاهرة، التحق نجيب الريحاني فرقة جورج أبيض، الذي كان قد ضمَّ - قُبيل ذلك - فرقته إلى فرقة سلامة حجازي . و منها ذاع صيته",
    "في استاد القاهرة، قام حفل افتتاح بطولة كأس الأمم الأفريقية بحضور رئيس الجمهورية و رئيس الاتحاد الدولي لكرة القدم",
    "من فضلك أرسل هذا البريد الى صديقي جلال الدين في تمام الساعة الخامسة صباحا في يوم الثلاثاء القادم",
    "امبارح اتفرجت على مباراة مانشستر يونايتد مع ريال مدريد في غياب الدون كرستيانو رونالدو",
    "لا تنسى تصحيني الساعة سبعة, و ضيف في الجدول اني احضر مباراة نادي النصر غدا",
]

for sample in samples:
    ents = _extract_ner(text=sample, model=model, tokenizer=tokenizer, start_token="▁")

    print(sample)
    for ent in ents:
        print("\t",ent["token"],"==>",ent["label"])
    print("========\n")

تلقى تعليمه في الكتاب ثم انضم الى الأزهر عام 1873م. تعلم على يد السيد جمال الدين الأفغاني والشيخ محمد عبده
	 عام 1873م. ==> time
	 السيد جمال الدين الأفغاني ==> person
	 محمد عبده ==> person

بعد عودته إلى القاهرة، التحق نجيب الريحاني فرقة جورج أبيض، الذي كان قد ضمَّ - قُبيل ذلك - فرقته إلى فرقة سلامة حجازي . و منها ذاع صيته
	 القاهرة، ==> location
	 نجيب الريحاني ==> person
	 فرقة جورج أبيض، ==> organization
	 فرقة سلامة حجازي ==> organization

في استاد القاهرة، قام حفل افتتاح بطولة كأس الأمم الأفريقية بحضور رئيس الجمهورية و رئيس الاتحاد الدولي لكرة القدم
	 استاد القاهرة، ==> location
	 بطولة كأس الأمم الأفريقية ==> event
	 رئيس الجمهورية ==> job
	 رئيس ==> job
	 الاتحاد الدولي لكرة القدم ==> organization

من فضلك أرسل هذا البريد الى صديقي جلال الدين في تمام الساعة الخامسة صباحا في يوم الثلاثاء القادم
	 جلال الدين ==> person
	 الساعة الخامسة صباحا ==> time
	 يوم الثلاثاء القادم ==> time

امبارح اتفرجت على مباراة مانشستر يونايتد مع ريال مدريد في غياب الدون كرستيانو رونالدو
	 مانشستر يو

In [7]:
import pandas as pd
import os

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# read from drive
n=500
MOKHBIR_DIR = "/content/drive/My Drive/NLP_PROJECT/Al_Mokhbir_Al_Eqtisadi/"
DA7EE7_DIR = "/content/drive/My Drive/NLP_PROJECT/Da7ee7/"
HADARAA_DIR = "/content/drive/My Drive/NLP_PROJECT/Fi_Al_Hadaraa/"

In [30]:
def data_loader(dir_path: str, n: int, read_metadata: bool = False) -> pd.DataFrame:
    """
    Load episode transcripts from text files and optionally add metadata.

    :param dir_path: Path to the directory containing 'raw_data' and 'metadata' folders.
    :param n: Number of episodes to read.
    :param read_metadata: Whether to include metadata from JSON files.
    :return: A pandas DataFrame.
    """
    data_dir_path = os.path.join(dir_path, "raw_data")
    metadata_dir_path = os.path.join(dir_path, "metadata") if read_metadata else None

    episodes = []

    # Read the first `n` .txt files
    txt_files = [file for file in os.listdir(data_dir_path) if file.endswith(".txt")][:n]

    for file in txt_files:
        base_name = file.replace(".txt", "")
        file_path = os.path.join(data_dir_path, file)

        with open(file_path, 'r', encoding="utf-8") as f:
            episode_data = {"file_name": base_name, "episode_transcript": f.read().strip()}

        # Read metadata if needed
        if read_metadata:
            metadata_path = os.path.join(metadata_dir_path, base_name + ".json")
            if os.path.exists(metadata_path):
                with open(metadata_path, 'r', encoding="utf-8") as f:
                    episode_data["length"] = json.load(f).get("length", None)
            else:
                episode_data["length"] = None  # Handle missing metadata

        episodes.append(episode_data)

    return pd.DataFrame(episodes)

In [31]:
mokhbir_df = data_loader(MOKHBIR_DIR, n, read_metadata=True)
da7ee7_df = data_loader(DA7EE7_DIR, n, read_metadata=True)
hadaraa_df = data_loader(HADARAA_DIR, n, read_metadata=True)
mokhbir_df.info(), da7ee7_df.info(), hadaraa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   file_name           222 non-null    object
 1   episode_transcript  222 non-null    object
 2   length              222 non-null    object
dtypes: object(3)
memory usage: 5.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145 entries, 0 to 144
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   file_name           145 non-null    object
 1   episode_transcript  145 non-null    object
 2   length              0 non-null      object
dtypes: object(3)
memory usage: 3.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   file_name           59 non-nu

(None, None, None)

In [32]:
mokhbir_df.head()

,file_name,episode_transcript,length
0,المخبر_الاقتصادي__كيف_تحدت_الصين_أمريكا_وسرقت...,الحقونا\n الصينيين سرقونا\nدا باختصار هو ملخص ...,00:18:18
1,المخبر_الاقتصادي___لماذا_ترفض_شركات_النفط_الأ...,في 9 ديسمبر 2022\nالرئيس الروسي فلاديمير بوتين...,00:16:23
2,المخبر_الاقتصادي___كيف_تخطط_أمريكا_لمنع_تطور_...,من حوالي السنتين\nوتحديداً في 3 ديسمبر 2020\nج...,00:14:52
3,المخبر_الاقتصادي__كيف_تستخدم_إسرائيل_نظام_ال...,بعد ساعات من انطلاق عمليه طوفان الاقصى\nيوم 7 ...,00:17:05
4,المخبر_الاقتصادي__هل_تستطيع_أقوى_امرأة_في_أ...,في مساء يوم 17 أكتوبر سنة 1989 \nزلزال عنيف بق...,00:17:43


In [33]:
da7ee7_df.head()

,file_name,episode_transcript,length
0,شاورما الفراخ الدحيح,9.84: لو سمحت\n11.01: [موسيقى]\n18.119: اعمل\n...,None
1,كريستيانو رونالدو - الجزء الأول الدحيح,"4.117: شايف مُلخّص لمسات الكابتن ""أشرف""؟\n6.71...",None
2,كلوب الدحيح,"0.0: ""كلوب""، حبيبي،\n1.593: احنا عايزين مصلحتك...",None
3,ماذا قال الدحيح عن مقاييس الجمال و الأتيكيت؟ ...,4.95: [موسيقى]\n59.42: [موسيقى]\n68.39: [موسيق...,None
4,ليه بنتخن في رمضان؟ الدحيح,0.0: أهلًا بيكم في حلقة جديدة،\nمن برنامجي أنا...,None


In [34]:
hadaraa_df.head()

,file_name,episode_transcript,length
0,الزواج_عن_حب_أم_زواج_الصالونات_.._هل_ظلمنا_سي...,"""لمّا تقرّب...\nأنا بأتونّس بيك\nوإمّا بتبعد.....",00:17:56
1,الإنسان_عامل_كام__في_الحضارة,آه، انت كدا يعني، ناوي تخبّي عليها؟\nلا، لا، ل...,00:12:22
2,البيت__أين_نجده__في_الحضارة,"يا ""ريم""، أنا أبعد ما يكون\nعن إني أنا أشتري ش...",00:15:33
3,صديقي_الإنسان_بتحب_الصدر_ولا_الورك__في_الحضارة,طب ايه\nالليل بيخلص وكاتب التاريخ بيغلق صفحاته...,00:17:09
4,سلطة_السخرية_.._إلى_أي_مدى_يمكننا_أن_نسخر_م...,هو كان لازم يعني تتلمُّوا عندي في الشقة؟!\nما ...,00:17:26


In [35]:
def add_ner_column(df:pd.DataFrame):
  df["ner"] = df["episode_transcript"].apply(lambda x: _extract_ner(text=x, model=model, tokenizer=tokenizer))
  return df

In [45]:
add_ner_column(hadaraa_df)
hadaraa_df.head()

,file_name,episode_transcript,length,ner
0,الزواج_عن_حب_أم_زواج_الصالونات_.._هل_ظلمنا_سي...,"""لمّا تقرّب...\nأنا بأتونّس بيك\nوإمّا بتبعد.....",00:17:56,"[{'token': 'عليا', 'label': 'person'}, {'token..."
1,الإنسان_عامل_كام__في_الحضارة,آه، انت كدا يعني، ناوي تخبّي عليها؟\nلا، لا، ل...,00:12:22,"[{'token': 'ساعة الصفا', 'label': 'event'}, {'..."
2,البيت__أين_نجده__في_الحضارة,"يا ""ريم""، أنا أبعد ما يكون\nعن إني أنا أشتري ش...",00:15:33,[]
3,صديقي_الإنسان_بتحب_الصدر_ولا_الورك__في_الحضارة,طب ايه\nالليل بيخلص وكاتب التاريخ بيغلق صفحاته...,00:17:09,"[{'token': '2007', 'label': 'time'}, {'token':..."
4,سلطة_السخرية_.._إلى_أي_مدى_يمكننا_أن_نسخر_م...,هو كان لازم يعني تتلمُّوا عندي في الشقة؟!\nما ...,00:17:26,[]


In [49]:
import os
import json
import pandas as pd

def append_ner_to_metadata(dir_path,df: pd.DataFrame):
    # Ensure 'dir_path' and 'ner' columns exist in DataFrame
    if "file_name" not in df.columns or "ner" not in df.columns:
        raise KeyError("DataFrame must contain 'file_path' and 'ner' columns.")

    for _, row in df.iterrows():
        base_name = "metadata/" + row["file_name"]
        file_path = os.path.join(dir_path, base_name + ".json")

        if not os.path.exists(file_path):
            print(f"Warning: File {file_path} not found. Skipping.")
            continue

        try:
            with open(file_path, 'r+', encoding="utf-8") as f:
                metadata = json.load(f)
                metadata["ner"] = row["ner"]

                # Move cursor to beginning before overwriting
                f.seek(0)
                json.dump(metadata, f, ensure_ascii=False, indent=2)
                f.truncate()
        except (json.JSONDecodeError, IOError) as e:
            print(f"Error processing {file_path}: {e}")

In [50]:
append_ner_to_metadata(HADARAA_DIR,hadaraa_df)

In [51]:
add_ner_column(mokhbir_df)
append_ner_to_metadata(MOKHBIR_DIR,mokhbir_df)

In [52]:
add_ner_column(da7ee7_df)
append_ner_to_metadata(DA7EE7_DIR,da7ee7_df)